In [ ]:
import os
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles

In [ ]:
folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
seed = 'rvs' #set seedlabel
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
script_dir = Path(f'/home/yufengxia/Code/yfXRepo/CCNN-23BPhappy/fMRI_Proc/')
script_dir = Path(script_dir)

for folderofsubj in folders:
    subj = folderofsubj[-4:]

    ppi_dir = Path(f'{folderofsubj}/funcppi/')
    ppi_dir = Path(ppi_dir)
    #assert not GLM_dir.exists(), f'GLM dir {GLM_dir} already exists'
    #verify that the results directory does not yet exist
    if not os.path.exists(ppi_dir):
        os.makedirs(ppi_dir)

    os.chdir(ppi_dir)   

    %cd $ppi_dir
    #generate seed time seires by errts
    ! echo 10 12 -8 | 3dUndump -xyz -srad 6 -master ../funcGLM1/{subj}_{Param1}_{Param2}_stats+tlrc  -prefix mask.{subj}.{seed}.nii.gz -
    ! 3dmaskave -quiet -mask mask.{subj}.{seed}.nii.gz ../funcGLM1/{subj}_{Param1}_{Param2}_errts+tlrc > ppi.{subj}.{seed}.1D
    #copy timing files into stimuli directory
    %cd /home/data/yufengspace/23BPhappy/parametric_modulation/
    ! mkdir -p $ppi_dir/stimuli
    ! cp -r {subj}_{Param1}*.txt $ppi_dir/stimuli/AFNI_timing.times.1.txt
    ! cp -r {subj}_{Param2}*.txt $ppi_dir/stimuli/AFNI_timing.times.2.txt

    %cd $ppi_dir
    # generate PPI regressors from seed and timing files
    ! tcsh $script_dir/scripts/cmd.ppi.23BPhappy.regs {subj} {seed}
    # copy the results into the stimuli directory
    ! cp work.{subj}/p6.* ppi.{subj}.{seed}.1D stimuli 
    # downsample 
    ! tcsh $script_dir/scripts/proc.3dd.ppi.post.full {subj} {seed}

In [ ]:
folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    ppi_dir = Path(f'{folderofsubj}/funcppi/')
    ppi_dir = Path(ppi_dir)
    # verify that the results directory does not yet exist
    ! rm -rf {ppi_dir}
